In [1]:
from selenium import webdriver
# encoding=utf8 
import sys
reload(sys) 
sys.setdefaultencoding('utf8')
#ASCII 编码无法对 unicode 的中文进行编码,因此就报错了。

In [2]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait

In [3]:
from selenium.webdriver.support import expected_conditions as EC
from stanfordcorenlp import StanfordCoreNLP
from pyquery import PyQuery as pq
import requests

In [5]:
from selenium.webdriver.common.keys import Keys
from nltk.corpus import PlaintextCorpusReader
from nltk import word_tokenize


In [6]:
corpus_root='/Users/jiweilu/Desktop/'
files=PlaintextCorpusReader(corpus_root,'cidian.txt')
files.fileids()#找到辞典文件

['cidian.txt']

In [7]:
words=word_tokenize(files.raw(fileids=files.fileids()))
words
#打印出书法辞典的内容

[u'\u4e00', u'\u4e8c', u'\u4e09', u'\u56db', u'\u4e94']

In [9]:
#遍历每个字，并依次对该字进行检索
for word in words:
    # 创建浏览器驱动对象
    option = webdriver.ChromeOptions()
    option.add_argument("headless")#chrome浏览器无界面运行
    driver = webdriver.Chrome(chrome_options=option)
    driver.get('http://shufa.guoxuedashi.com/')
    
    # 显式等待，设置timeout
    wait = WebDriverWait(driver, 9)
    input = wait.until(
    EC.presence_of_element_located(
        (By.CSS_SELECTOR, '#sokeyshufa')))
    #配合该类的until()和until_not()方法，就能够根据判断条件而进行灵活地等待了。 
    #程序每隔9秒看一眼，如果条件成立了，则执行下一步，否则继续等待，直到超过设置的最长时间抛出TimeoutException。
    
    
    driver.find_element_by_id("sokeyshufa").send_keys(word)#发送当前遍历的书法字
    driver.find_element_by_xpath("//div[@id='oneCon8']//input[@name='submit']").click()#模拟点击
#之前之所以用webdriver是因为需要模拟点击事件。之后都采用request

    path=driver.current_url#获取当前页的url（即书法字的页面）
    r= requests.get(path)
    html = r.content
    doc = pq(html)#文件初始化
    doc('.table2')#找到class为table2的元素
    #len(doc('.table2'))
    for doc2 in doc('.table2'):
        table2_0 = doc2
        pq_table2_0 = pq(table2_0)
        pq_table2_0('a')#找到包含超链接的元素
        #len(pq_table2_0('a'))
        for pq_table20 in pq_table2_0('a'):
            pq_a = pq(pq_table20)
            pq_a('img').attr('src')
            url=pq_a('img').attr('src')#找到当前图片的url
            filename = '/Users/jiweilu/Desktop/pic.txt'
            with open(filename,'a') as f: # 'a'表示append,即在原来文件内容后继续写数据（不清楚原有数据）
                f.write("\n"+word+":\n"+url)
    

In [10]:
# 关闭浏览器驱动
driver.close()